In [1]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import datetime
import numpy as np
import psd_tool
import data_loader
from field_models import model
import matplotlib.pyplot as plt

SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import importlib
importlib.reload(psd_tool)
importlib.reload(data_loader)

%matplotlib qt

This unreleased version of SpacePy is not supported by the SpacePy team.


In [2]:
start = datetime.datetime(year = 2017,
                            month = 7,
                            day = 21)

end = datetime.datetime(year = 2017, 
                        month = 7, 
                        day = 27)

satellite = "A"
chosen_mu = 1000
chosen_k = 0.05
model = model.TS04D

refs = data_loader.load_psd(satellite=satellite, field_model=model, start=start, end=end)

selected_refs = psd_tool.select_mu_and_k_from_psd(refs=refs, chosen_mu=chosen_mu, chosen_k=chosen_k)

epoch = selected_refs["EPOCH"]
Lstar = selected_refs["L_STAR"] 
L = selected_refs["L"]
PSD = selected_refs["PSD"]
in_out = selected_refs["IN_OUT"]
orbit_number = selected_refs["ORBIT_NUMBER"]

not_nan = np.isfinite(PSD)
epoch = epoch[not_nan]
Lstar = Lstar[not_nan]
L = L[not_nan]
PSD = PSD[not_nan]
in_out = in_out[not_nan]
orbit_number = orbit_number[not_nan]


2017-07-01 00:00:00
Loading : PSD_201707_A_TS04D.npz
Time taken for loop: 2.1037793159484863


In [3]:
min_lstar = 2.0
max_lstar = 6.0

unique_orbits = np.unique(orbit_number)
start_of_orbits = []

fig, ax = plt.subplots(1,1)

for orbit in unique_orbits:
    
    inbound = (orbit_number == orbit) & (in_out == -1)
    outbound = (orbit_number == orbit) & (in_out == 1)
    
    if np.any(np.nonzero(inbound)[0]):
        
        start_of_inbound_orbit = epoch[np.nonzero(inbound)[0][0]]
        
        binned_Lstar, binned_PSD = psd_tool.bin_radial_profile(LSTAR = Lstar[inbound],
                                                               PSD = PSD[inbound],
                                                               LSTAR_MIN = min_lstar,
                                                               LSTAR_MAX = max_lstar,
                                                               dL = 0.10)
        
        psd_tool.plot_radial_profile(LSTAR = binned_Lstar,
                                     PSD = binned_PSD,
                                     IS_INBOUND = True,
                                     START_OF_ORBIT = start_of_inbound_orbit)
        
        start_of_orbits.append(start_of_inbound_orbit)

    if np.any(np.nonzero(outbound)[0]):
        
        start_of_outbound_orbit = epoch[np.nonzero(outbound)[0][0]]
        
        binned_Lstar, binned_PSD = psd_tool.bin_radial_profile(LSTAR = Lstar[outbound],
                                                               PSD = PSD[outbound],
                                                               LSTAR_MIN = min_lstar,
                                                               LSTAR_MAX = max_lstar,
                                                               dL = 0.10)
        
        psd_tool.plot_radial_profile(LSTAR = binned_Lstar,
                                     PSD = binned_PSD,
                                     IS_INBOUND = False,
                                     START_OF_ORBIT = start_of_outbound_orbit)
        
        start_of_orbits.append(start_of_outbound_orbit)

ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
ax.xaxis.grid(color='gray', linestyle='dashed')
plt.xlabel("L*")
plt.ylabel("PSD $(c/MeV/cm)^{3}$")

plt.xlim((min_lstar, max_lstar))

handles, labels = plt.gca().get_legend_handles_labels()
order = np.argsort(start_of_orbits)
colors = plt.get_cmap("viridis", len(order)) # Initialize holder for trajectories
j = 0
for i in order:
    handles[i].set_color(colors(j))
    j += 1

plt.legend([handles[i] for i in order], [labels[i] for i in order])
plt.title(rf"RBSP-{satellite.upper()}, $\mu$ = {chosen_mu} (MeV / G), K = {chosen_k} ($R_E$$G^{{1/2}}$), {model.name}")
fig.set_size_inches(12, 8)
#plt.savefig((OUTPUT_DIR + f"{year}-{month}-{day}-RBSP-{sat.upper()}-Mu-{int(CHOSEN_MU)}-K-{int(CHOSEN_K * 100)}-{FIELD_MODEL.name}.pdf"), format="pdf", dpi=100)

plt.show()